In [ ]:
import csv
import sys

ASFILE = "../usb/ovk/data/eriktks/AS/text/AS-mails.csv"
ESFILE = "../usb/ovk/data/eriktks/ES/text/ES-mails.csv"
SEPARATOR = ","
CLIENT = "client-id"
COUNSELOR = "counselor"

In [ ]:
OVKMETAFILE = "../usb/ovk/data/eriktks/spss/opve.csv"
IDFIELDNAME = "onderzoeksnummer1"
EXITFIELDNAME = "Redenstoppen"

def readMetaData():
    exitData = {}
    cesdDiff = {}
    mhcDiff = {}
    stopReasons = {}
    try: inFile = open(OVKMETAFILE,"r")
    except Exception as e: sys.exit("cannot read file "+OVKMETAFILE+": "+str(e))
    csvReader = csv.DictReader(inFile,delimiter=SEPARATOR)
    for row in csvReader: 
        exitData[row[IDFIELDNAME]] = row[EXITFIELDNAME].strip()
        if exitData[row[IDFIELDNAME]]:
            stopReason = exitData[row[IDFIELDNAME]].lower()
            if not stopReason in stopReasons: stopReasons[stopReason] = 0
            stopReasons[stopReason] += 1
        if row["CESD_TOT_t0"] != "NA" and row["CESD_TOT_t1"] != "NA": 
            cesdDiff[row[IDFIELDNAME]] = int(row["CESD_TOT_t1"])-int(row["CESD_TOT_t0"])
        if row["MHCtot_t0"] != "NA" and row["MHCtot_t1"] != "NA": 
            mhcDiff[row[IDFIELDNAME]] = float(row["MHCtot_t1"])-float(row["MHCtot_t0"])
    inFile.close()
    return(exitData,cesdDiff,mhcDiff,stopReasons)

exitData,cesdDiff,mhcDiff,stopReasons = readMetaData()

Reasons for stopping: different expectations (6), personal problems (4), external factors (3), other (1), unknown (12)

In [ ]:
def testFunction(clientId):
    return(combi3(clientId))

def betterMental(clientId):
    if clientId == "stopped": return("decreased")
    elif clientId == "name": return("MHC")
    elif not clientId in mhcDiff: return(None)
    else: return(mhcDiff[clientId] > 0)

def lessDepressed(clientId):
    if clientId == "stopped": return("increased")
    elif clientId == "name": return("CESD")
    elif not clientId in cesdDiff: return(None)
    else: return(cesdDiff[clientId] < 0)
    
def finished(clientId):
    if clientId == "stopped": return("stopped")
    elif clientId == "name": return("Treatment progress")
    elif not clientId in exitData: return(None)
    else: return(exitData[clientId] == "")

def combi3(clientId):
    if clientId == "stopped": return("failed one test")
    elif clientId == "name": return("combi3")
    bm = betterMental(clientId)
    ld = lessDepressed(clientId)
    fi = finished(clientId)
    if bm == None or ld == None or fi == None: return(None)
    else: return(bm and ld and fi)

def readData():
    clientMails = {"unknown":{},"finished":{},"stopped":{}}
    counselorMails = {"unknown":{},"finished":{},"stopped":{}}
    counselorClients = {"unknown":{},"finished":{},"stopped":{}}
    for inFileName in [ASFILE,ESFILE]:
        try: inFile = open(inFileName,"r")
        except Exception as e: sys.exit(str(e)+" Cannot read file "+inFileName+": "+str(e))
        csvReader = csv.DictReader(inFile,delimiter=SEPARATOR)
        for row in csvReader:
            try:
                client = row[CLIENT]
                counselor = row[COUNSELOR]
                if testFunction(client) == None: treatmentStatus = "unknown"
                elif testFunction(client): treatmentStatus = "finished"
                else: treatmentStatus = "stopped"
                if not client in clientMails[treatmentStatus]: 
                    clientMails[treatmentStatus][client] = 0
                clientMails[treatmentStatus][client] += 1
                if not counselor in counselorMails[treatmentStatus]: 
                    counselorMails[treatmentStatus][counselor] = 0
                counselorMails[treatmentStatus][counselor] += 1
                if not counselor in counselorClients[treatmentStatus]: 
                    counselorClients[treatmentStatus][counselor] = {}
                if not client in counselorClients[treatmentStatus][counselor]: 
                    counselorClients[treatmentStatus][counselor][client] = True
            except Exception as e: sys.exit(str(e)+" Unexpected row in file "+inFileName+": "+str(row))
        inFile.close()
    return(clientMails,counselorMails,counselorClients)

def fillEmptySpots(myDict,filler):
    for ts1 in myDict:
        for key in myDict[ts1]:
            for ts2 in myDict:
                if ts2 != ts1 and not key in myDict[ts2]:
                    myDict[ts2][key] = filler
    return(myDict)

clientMails,counselorMails,counselorClients = readData()
counselorMails = fillEmptySpots(counselorMails,0)
counselorClients = fillEmptySpots(counselorClients,{})

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

COLORS = { "unknown":"grey","finished":"blue","stopped":"red"}

def computeAverage(myDict):
    myValues = {}
    for key1 in myDict:
        for key2 in myDict[key1]:
            if not key2 in myValues: myValues[key2] = myDict[key1][key2]
            else: myValues[key2] += myDict[key1][key2]
    return(np.average(list(myValues.values())))

def computeMinMax(myDict):
    myMin = None
    myMax = None
    for key in myDict:
        if len(myDict[key]) > 0:
            minList = min(myDict[key].keys())
            maxList = max(myDict[key].keys())
            if myMin == None or myMin > minList: myMin = minList
            if myMax == None or myMax < maxList: myMax = maxList
    return(myMin,myMax)

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(2,1,1)
nbrOfClients = np.sum([len(clientMails[x]) for x in clientMails])
average = computeAverage(clientMails)
plt.title("OVK data: "+testFunction("name")+". Number of mails per client: "+str(nbrOfClients)+" clients; "+
          str(len(clientMails["stopped"]))+" "+testFunction("stopped")+"; "+str(len(clientMails["unknown"]))+
          " unknown; average number of mails per client: "+str(int(average)))
for ts in clientMails:
    plt.bar([int(x) for x in clientMails[ts].keys()],[x for x in clientMails[ts].values()],color=COLORS[ts])
myMin,myMax = computeMinMax(clientMails)
plt.axis([7995,8288,0,40])
plt.plot((int(myMin),int(myMax)),(average,average),"y")

plt.subplot(2,2,3)
average = computeAverage(counselorMails)
plt.title("Mails per counselor (average: "+str(int(average))+")")
countsUnknown = [x for x in counselorMails["unknown"].values()]
countsStopped = [x for x in counselorMails["stopped"].values()]
countsFinished = [x for x in counselorMails["finished"].values()]
countsUnknownStopped = np.add(countsUnknown,countsStopped)
plt.bar([x for x in range(1,len(countsUnknown)+1)],countsUnknown,color=COLORS["unknown"])
plt.bar([x for x in range(1,len(countsStopped)+1)],countsStopped,color=COLORS["stopped"],bottom=countsUnknown)
plt.bar([x for x in range(1,len(countsFinished)+1)],countsFinished,color=COLORS["finished"],
        bottom=countsUnknownStopped)
plt.axis([0.5,9.5,0,400])
#plt.bar([i for i in range(0,len(counselorMails["finished"]))],[x for x in counselorMails["finished"].values()])
plt.plot((0.5,[len(counselorMails[x]) for x in counselorMails][0]+0.5),(average,average),"y")

plt.subplot(2,2,4)
countsUnknown = [len(x) for x in counselorClients["unknown"].values()]
countsStopped = [len(x) for x in counselorClients["stopped"].values()]
countsFinished = [len(x) for x in counselorClients["finished"].values()]
countsUnknownStopped = np.add(countsUnknown,countsStopped)
average = np.average(np.add(countsUnknown,np.add(countsStopped,countsFinished)))
plt.title("Clients per counselor (average: "+str(int(average))+")")
plt.bar([x for x in range(1,len(countsUnknown)+1)],countsUnknown,color=COLORS["unknown"])
plt.bar([x for x in range(1,len(countsStopped)+1)],countsStopped,color=COLORS["stopped"],bottom=countsUnknown)
plt.bar([x for x in range(1,len(countsFinished)+1)],countsFinished,color=COLORS["finished"],
        bottom=countsUnknownStopped)
plt.plot((0.5,[len(counselorClients[x]) for x in counselorClients][0]+0.5),(average,average),"y")
plt.axis([0.5,9.5,0,23])
plt.show()

Note: for two clients that stopped the treatment according to the metadata (8068, 8077), there are no mails in the collection. The mails of the client that was banned from the treatment (8236), were not used. 

In [ ]:
bmld = {}
ldfi = {}
fibm = {}
for ts in counselorClients:
    for cs in counselorClients[ts]:
        for cl in counselorClients[ts][cs]:
            bm = betterMental(cl)
            ld = lessDepressed(cl)
            fi = finished(cl)
            if not bm in bmld: bmld[bm] = {}
            if not ld in bmld[bm]: bmld[bm][ld] = 0
            bmld[bm][ld] += 1
            if not ld in ldfi: ldfi[ld] = {}
            if not fi in ldfi[ld]: ldfi[ld][fi] = 0
            ldfi[ld][fi] += 1
            if not fi in fibm: fibm[fi] = {}
            if not bm in fibm[fi]: fibm[fi][bm] = 0
            fibm[fi][bm] += 1
print("bm-ld")
for bm in bmld.keys():
    for ld in bmld[bm]:
        print(bm,ld,bmld[bm][ld],end="#")
    print("")
print("ld-fi")
for ld in ldfi.keys():
    for fi in ldfi[ld]:
        print(ld,fi,ldfi[ld][fi],end="#")
    print("")
print("fi-bm")
for fi in fibm.keys():
    for bm in fibm[fi]:
        print(fi,bm,fibm[fi][bm],end="#")
    print("")
print("")
print("BM=True | LD=True: ",bmld[True][True]/(bmld[None][True]+bmld[True][True]+bmld[False][True]))
print("BM=False | LD=False: ",bmld[False][False]/(0+bmld[True][False]+bmld[False][False]))
print("LD=True | BM=True: ",bmld[True][True]/(0+bmld[True][True]+bmld[True][False]))
print("LD=False | BM=False: ",bmld[False][False]/(0+bmld[False][True]+bmld[False][False]))

print("LD=True | FI=True: ",ldfi[True][True]/(ldfi[None][True]+ldfi[True][True]+ldfi[False][True]))
print("LD=False | FI=False: ",ldfi[False][False]/(ldfi[None][False]+ldfi[True][False]+ldfi[False][False]))
print("FI=True | LD=True: ",ldfi[True][True]/(0+ldfi[True][True]+ldfi[True][False]))
print("FI=False | LD=False: ",ldfi[False][False]/(0+ldfi[False][True]+ldfi[False][False]))

print("FI=True | BM=True: ",fibm[True][True]/(0+fibm[True][True]+fibm[False][True]))
print("FI=False | BM=False: ",fibm[False][False]/(0+fibm[True][False]+fibm[False][False]))
print("BM=True | FI=True: ",fibm[True][True]/(fibm[True][None]+fibm[True][True]+fibm[True][False]))
print("BM=False | FI=False: ",fibm[False][False]/(fibm[False][None]+fibm[False][True]+fibm[False][False]))